PWC-Net-small model finetuning (with cyclical learning rate schedule)
=======================================================

In this notebook we:
- Use a small model (no dense or residual connections), 6 level pyramid, uspample level 2 by 4 as the final flow prediction
- Train the PWC-Net-small model on a mix of the `FlyingChairs` and `FlyingThings3DHalfRes` dataset using a Cyclic<sub>short</sub> schedule of our own
- Let the Cyclic<sub>short</sub> schedule oscillate between `2e-05` and `1e-06` for 200,000 steps
- Switch to the "robust" loss described in the paper, instead of the "multiscale" loss used during training

Below, look for `TODO` references and customize this notebook based on your own needs.

## Reference

[2018a]<a name="2018a"></a> Sun et al. 2018. PWC-Net: CNNs for Optical Flow Using Pyramid, Warping, and Cost Volume. [[arXiv]](https://arxiv.org/abs/1709.02371) [[web]](http://research.nvidia.com/publication/2018-02_PWC-Net%3A-CNNs-for) [[PyTorch (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/PyTorch) [[Caffe (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/Caffe)

In [1]:
"""
pwcnet_finetune.ipynb

PWC-Net model finetuning.

Written by Phil Ferriere

Licensed under the MIT License (see LICENSE for details)

Tensorboard:
    [win] tensorboard --logdir=E:\\repos\\tf-optflow\\tfoptflow\\pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned
    [ubu] tensorboard --logdir=/media/EDrive/repos/tf-optflow/tfoptflow/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned
"""
from __future__ import absolute_import, division, print_function
import sys
from copy import deepcopy
from dataset_mpisintel import MPISintelDataset
from dataset_base import _DEFAULT_DS_TUNE_OPTIONS
from dataset_flyingchairs import FlyingChairsDataset
from dataset_flyingthings3d import FlyingThings3DHalfResDataset
from dataset_kitti import KITTIDataset
from dataset_mixer import MixedDataset
from model_pwcnet import ModelPWCNet, _DEFAULT_PWCNET_FINETUNE_OPTIONS

In [2]:
!nvidia-smi

Mon Jun  3 19:53:32 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.40.04    Driver Version: 418.40.04    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           Off  | 00000000:04:00.0 Off |                  Off |
| N/A   36C    P0    49W / 250W |    509MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P40           Off  | 00000000:0D:00.0 Off |                  Off |
| N/A   25C    P8     9W / 250W |      0MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

## TODO: Set this first!

In [3]:
# TODO: You MUST set dataset_root to the correct path on your machine!
if sys.platform.startswith("win"):
    _DATASET_ROOT = 'E:/datasets/'
else:
    _DATASET_ROOT = '/Vol1/dbstore/datasets/KITTI_odometry_2012/' #'/Vol1/dbstore/datasets/sintel/'

    
# TODO: You MUST adjust the settings below based on the number of GPU(s) used for training
# Set controller device and devices
# A one-gpu setup would be something like controller='/device:GPU:0' and gpu_devices=['/device:GPU:0']
# Here, we use a dual-GPU setup, as shown below
# gpu_devices = ['/device:GPU:0', '/device:GPU:1']
# controller = '/device:CPU:0'
gpu_devices = ['/device:GPU:0']
controller = '/device:GPU:0'

# TODO: You MUST adjust this setting below based on the amount of memory on your GPU(s)
# Batch size
batch_size = 2

# Finetune on `sintel` mix

## Load the dataset

In [4]:
# TODO: You MUST set the batch size based on the capabilities of your GPU(s) 
#  Load train dataset
ds_opts = deepcopy(_DEFAULT_DS_TUNE_OPTIONS)
ds_opts['in_memory'] = False                          # Too many samples to keep in memory at once, so don't preload them
ds_opts['aug_type'] = 'basic'                         # Apply all supported augmentations
ds_opts['batch_size'] = batch_size * len(gpu_devices) # Use a multiple of 8; here, 16 for dual-GPU mode (Titan X & 1080 Ti)
#ds1 = FlyingChairsDataset(mode='train_with_val', ds_root=_FLYINGCHAIRS_ROOT, options=ds_opts)
ds_opts['type'] = 'occ'

#ds2 = FlyingThings3DHalfResDataset(mode='train_with_val', ds_root=_FLYINGTHINGS3DHALFRES_ROOT, options=ds_opts)
#ds = MixedDataset(mode='train_with_val', datasets=[ds1, ds2], options=ds_opts)

In [5]:
ds_opts['crop_preproc'] = (320,960)

In [6]:
# ds = MPISintelDataset(mode='train_with_val', ds_root=_MPISINTEL_ROOT, options=ds_opts)

In [7]:
ds = KITTIDataset(mode = 'train_with_val', ds_root=_DATASET_ROOT, options=ds_opts)

In [8]:
# Display dataset configuration
ds.print_config()


Dataset Configuration:
  verbose              False
  in_memory            False
  crop_preproc         (320, 960)
  scale_preproc        None
  type                 occ
  tb_test_imgs         False
  random_seed          1969
  val_split            0.03
  aug_type             basic
  aug_labels           True
  fliplr               0.5
  flipud               0.5
  translate            (0.5, 0.05)
  scale                (0.5, 0.05)
  batch_size           2
  mode                 train_with_val
  train size           188
  val size             6


In [12]:
ckpt_path = './models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000'

## Configure the finetuning

In [13]:
# Start from the default options
nn_opts = deepcopy(_DEFAULT_PWCNET_FINETUNE_OPTIONS)
nn_opts['verbose'] = True
nn_opts['ckpt_path'] = './models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000'
nn_opts['ckpt_dir'] = './pwcnet-lg-6-2-cyclic-KITTI_finetuned/'
nn_opts['batch_size'] = 8
nn_opts['x_shape'] = [2, ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 3]
nn_opts['y_shape'] = [ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 2]
nn_opts['use_tf_data'] = True # Use tf.data reader
nn_opts['gpu_devices'] = gpu_devices
nn_opts['controller'] = controller

# Use the PWC-Net-small model in quarter-resolution mode
nn_opts['use_dense_cx'] = True
nn_opts['use_res_cx'] = True
nn_opts['pyr_lvls'] = 6
nn_opts['flow_pred_lvl'] = 2

# Robust loss as described doesn't work, so try the following:
nn_opts['loss_fn'] = 'loss_multiscale' # 'loss_multiscale' # 'loss_robust' # 'loss_robust'
nn_opts['q'] = 1. # 0.4 # 1. # 0.4 # 1.
nn_opts['epsilon'] = 0. # 0.01 # 0. # 0.01 # 0.

In [14]:
# Set the learning rate schedule. This schedule is for a single GPU using a batch size of 8.
# Below,we adjust the schedule to the size of the batch and the number of GPUs.
nn_opts['lr_policy'] = 'multisteps'
nn_opts['init_lr'] = 1e-05
nn_opts['lr_boundaries'] = [80000, 120000, 160000, 200000]
nn_opts['lr_values'] = [1e-05, 5e-06, 2.5e-06, 1.25e-06, 6.25e-07]
nn_opts['max_steps'] = 200000

# Below,we adjust the schedule to the size of the batch and our number of GPUs (2).
nn_opts['max_steps'] = int(nn_opts['max_steps'] * 8 / 8)
nn_opts['cyclic_lr_stepsize'] = int(nn_opts['cyclic_lr_stepsize'] * 8 / 8)

In [15]:
# Instantiate the model and display the model configuration
nn = ModelPWCNet(mode='train_with_val', options=nn_opts, dataset=ds)
nn.print_config()

Building model...
... model built.
Configuring training ops...
... training ops configured.
Initializing from pre-trained model at ./models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000 for finetuning...

INFO:tensorflow:Restoring parameters from ./models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000
... model initialized

Model Configuration:
  verbose                True
  ckpt_path              ./models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000
  ckpt_dir               ./pwcnet-lg-6-2-cyclic-KITTI_finetuned/
  max_to_keep            10
  x_dtype                <dtype: 'float32'>
  x_shape                [2, 320, 960, 3]
  y_dtype                <dtype: 'float32'>
  y_shape                [320, 960, 2]
  train_mode             fine-tune
  adapt_info             None
  sparse_gt_flow         False
  display_step           100
  snapshot_step          1000
  val_step               1000
  val_batch_size         -1
  tb_val_imgs            to

## Finetune the model

In [16]:
# Train the model
nn.train()

Start finetuning...
Instructions for updating:
Use `tf.data.experimental.shuffle_and_repeat(...)`.
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
2019-05-17 15:19:06 Iter 100 [Train]: loss=430.29, epe=12.96, lr=0.000010, samples/sec=6.8, sec/step=1.181, eta=2 days, 17:34:21
2019-05-17 15:20:43 Iter 200 [Train]: loss=381.94, epe=11.33, lr=0.000010, samples/sec=8.5, sec/step=0.946, eta=2 days, 4:28:52
2019-05-17 15:22:23 Iter 300 [Train]: loss=359.52, epe=10.63, lr=0.000010, samples/sec=8.3, sec/step=0.965, eta=2 days, 5:33:21
2019-05-17 15:24:00 Iter 400 [Train]: loss=348.23, epe=10.26, lr=0.000010, samples/sec=8.4, sec/step=0.950, eta=2 days, 4:38:42
2019-05-17 15:25:38 Iter 500 [Train]: loss=341.28, epe=10.07, lr=0.000010, samples/sec=8.4, sec/step=0.955, eta=2 days, 4:56:47
2019-05-17 15:27:16 Iter 600 [Train]: loss=335.96, epe=9.92, lr=0.000010, samples/sec=8.4, sec/step=0.956, eta=2 days, 4:57:30
2019-05-17 15:28:55 Iter 700 [Train]: loss=335.89, epe=9.92

/Vol0/user/f.konokhov/miniconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Vol0/user/f.konokhov/miniconda3/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcnet.ckpt-1000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcnet.ckpt-1000
2019-05-17 15:35:40 Iter 1100 [Train]: loss=315.77, epe=9.27, lr=0.000010, samples/sec=8.3, sec/step=0.963, eta=2 days, 5:12:34
2019-05-17 15:37:18 Iter 1200 [Train]: loss=313.01, epe=9.16, lr=0.000010, samples/sec=8.3, sec/step=0.960, eta=2 days, 5:01:47
2019-05-17 15:38:57 Iter 1300 [Train]: loss=313.06, epe=9.17, lr=0.000010, samples/sec=8.3, sec/step=0.960, eta=2 days, 4:59:56
2019-05-17 15:40:35 Iter 1400 [Train]: loss=305.13, epe=8.92, lr=0.000010, samples/sec=8.4, sec/step=0.956, eta=2 days, 4:44:29
2019-05-17 15:42:13 Iter 1500 [Train]: loss=302.68, epe=8.83, lr=0.000010, samples/sec=8.4, sec/step=0.950, eta=2 days, 4:22:17
2019-05-17 15:43:52 Iter 1600 [Train]: loss=302.69, epe=8.83, lr=0.000010, samples/sec=8.3, sec/step=0.967, eta=2 days, 5:16:08
2019-05-17 

2019-05-17 17:01:19 Iter 6300 [Train]: loss=272.32, epe=7.89, lr=0.000010, samples/sec=8.4, sec/step=0.949, eta=2 days, 3:02:33
2019-05-17 17:02:57 Iter 6400 [Train]: loss=271.81, epe=7.88, lr=0.000010, samples/sec=8.4, sec/step=0.953, eta=2 days, 3:14:22
2019-05-17 17:04:36 Iter 6500 [Train]: loss=273.18, epe=7.95, lr=0.000010, samples/sec=8.4, sec/step=0.957, eta=2 days, 3:27:37
2019-05-17 17:06:17 Iter 6600 [Train]: loss=273.05, epe=7.92, lr=0.000010, samples/sec=8.2, sec/step=0.978, eta=2 days, 4:32:36
2019-05-17 17:07:56 Iter 6700 [Train]: loss=269.22, epe=7.81, lr=0.000010, samples/sec=8.3, sec/step=0.964, eta=2 days, 3:45:29
2019-05-17 17:09:36 Iter 6800 [Train]: loss=270.83, epe=7.84, lr=0.000010, samples/sec=8.3, sec/step=0.968, eta=2 days, 3:56:03
2019-05-17 17:11:15 Iter 6900 [Train]: loss=270.82, epe=7.84, lr=0.000010, samples/sec=8.3, sec/step=0.959, eta=2 days, 3:24:55
2019-05-17 17:12:54 Iter 7000 [Train]: loss=270.93, epe=7.86, lr=0.000010, samples/sec=8.3, sec/step=0.9

2019-05-17 18:34:57 Iter 11700 [Train]: loss=263.08, epe=7.61, lr=0.000010, samples/sec=8.0, sec/step=1.003, eta=2 days, 4:29:05
2019-05-17 18:36:45 Iter 11800 [Train]: loss=264.85, epe=7.65, lr=0.000010, samples/sec=7.9, sec/step=1.012, eta=2 days, 4:53:53
2019-05-17 18:38:33 Iter 11900 [Train]: loss=257.09, epe=7.43, lr=0.000010, samples/sec=7.8, sec/step=1.023, eta=2 days, 5:25:33
2019-05-17 18:40:20 Iter 12000 [Train]: loss=258.51, epe=7.48, lr=0.000010, samples/sec=7.9, sec/step=1.012, eta=2 days, 4:52:11
2019-05-17 18:40:20 Iter 12000 [Val]: loss=nan, epe=nan
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcnet.ckpt-12000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcnet.ckpt-12000
2019-05-17 18:42:19 Iter 12100 [Train]: loss=256.86, epe=7.42, lr=0.000010, samples/sec=7.9, sec/step=1.019, eta=2 days, 5:09:47
2019-05-17 18:44:07 Iter 12200 [Train]: loss=257.38, epe=7.44, lr=0.000010, 

...

2019-05-19 18:02:09 Iter 177500 [Train]: loss=216.47, epe=6.24, lr=0.000001, samples/sec=7.7, sec/step=1.042, eta=6:30:47
2019-05-19 18:04:02 Iter 177600 [Train]: loss=218.95, epe=6.30, lr=0.000001, samples/sec=7.6, sec/step=1.047, eta=6:30:49
2019-05-19 18:05:50 Iter 177700 [Train]: loss=215.34, epe=6.20, lr=0.000001, samples/sec=8.0, sec/step=0.999, eta=6:11:09
2019-05-19 18:07:40 Iter 177800 [Train]: loss=214.90, epe=6.20, lr=0.000001, samples/sec=7.9, sec/step=1.016, eta=6:16:00
2019-05-19 18:09:37 Iter 177900 [Train]: loss=212.45, epe=6.12, lr=0.000001, samples/sec=7.6, sec/step=1.052, eta=6:27:37
2019-05-19 18:11:29 Iter 178000 [Train]: loss=217.46, epe=6.27, lr=0.000001, samples/sec=7.8, sec/step=1.029, eta=6:17:14
2019-05-19 18:11:29 Iter 178000 [Val]: loss=nan, epe=nan
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcnet.ckpt-178000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcne

Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcnet.ckpt-183000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcnet.ckpt-183000
2019-05-19 19:45:26 Iter 183100 [Train]: loss=216.54, epe=6.25, lr=0.000001, samples/sec=8.3, sec/step=0.961, eta=4:30:39
2019-05-19 19:47:05 Iter 183200 [Train]: loss=218.54, epe=6.31, lr=0.000001, samples/sec=8.5, sec/step=0.945, eta=4:24:41
2019-05-19 19:48:44 Iter 183300 [Train]: loss=212.63, epe=6.11, lr=0.000001, samples/sec=8.4, sec/step=0.956, eta=4:26:02
2019-05-19 19:50:24 Iter 183400 [Train]: loss=218.67, epe=6.32, lr=0.000001, samples/sec=8.3, sec/step=0.959, eta=4:25:20
2019-05-19 19:52:04 Iter 183500 [Train]: loss=214.88, epe=6.18, lr=0.000001, samples/sec=8.4, sec/step=0.958, eta=4:23:22
2019-05-19 19:53:43 Iter 183600 [Train]: loss=215.65, epe=6.22, lr=0.000001, samples/sec=8.4, sec/step=0.955, eta=4:21:08
2019-05-19 19:55:22 Iter 183700 [Train]: lo

2019-05-19 21:15:43 Iter 188500 [Train]: loss=212.62, epe=6.13, lr=0.000001, samples/sec=8.4, sec/step=0.957, eta=3:03:30
2019-05-19 21:17:23 Iter 188600 [Train]: loss=213.99, epe=6.18, lr=0.000001, samples/sec=8.3, sec/step=0.960, eta=3:02:23
2019-05-19 21:19:02 Iter 188700 [Train]: loss=215.22, epe=6.22, lr=0.000001, samples/sec=8.3, sec/step=0.960, eta=3:00:48
2019-05-19 21:20:41 Iter 188800 [Train]: loss=216.45, epe=6.25, lr=0.000001, samples/sec=8.4, sec/step=0.954, eta=2:58:02
2019-05-19 21:22:20 Iter 188900 [Train]: loss=216.49, epe=6.25, lr=0.000001, samples/sec=8.4, sec/step=0.948, eta=2:55:28
2019-05-19 21:23:59 Iter 189000 [Train]: loss=217.51, epe=6.28, lr=0.000001, samples/sec=8.4, sec/step=0.955, eta=2:55:07
2019-05-19 21:23:59 Iter 189000 [Val]: loss=nan, epe=nan
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcnet.ckpt-189000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcne

Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcnet.ckpt-194000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcnet.ckpt-194000
2019-05-19 22:49:14 Iter 194100 [Train]: loss=216.13, epe=6.23, lr=0.000001, samples/sec=8.3, sec/step=0.963, eta=1:34:40
2019-05-19 22:50:53 Iter 194200 [Train]: loss=213.06, epe=6.13, lr=0.000001, samples/sec=8.4, sec/step=0.957, eta=1:32:28
2019-05-19 22:52:33 Iter 194300 [Train]: loss=218.24, epe=6.30, lr=0.000001, samples/sec=8.3, sec/step=0.962, eta=1:31:24
2019-05-19 22:54:11 Iter 194400 [Train]: loss=217.82, epe=6.28, lr=0.000001, samples/sec=8.4, sec/step=0.950, eta=1:28:39
2019-05-19 22:55:50 Iter 194500 [Train]: loss=216.83, epe=6.25, lr=0.000001, samples/sec=8.4, sec/step=0.953, eta=1:27:21
2019-05-19 22:57:30 Iter 194600 [Train]: loss=212.90, epe=6.13, lr=0.000001, samples/sec=8.3, sec/step=0.962, eta=1:26:32
2019-05-19 22:59:09 Iter 194700 [Train]: lo

2019-05-20 00:19:25 Iter 199500 [Train]: loss=214.07, epe=6.17, lr=0.000001, samples/sec=8.4, sec/step=0.955, eta=0:07:57
2019-05-20 00:21:04 Iter 199600 [Train]: loss=219.84, epe=6.33, lr=0.000001, samples/sec=8.4, sec/step=0.954, eta=0:06:22
2019-05-20 00:22:43 Iter 199700 [Train]: loss=214.61, epe=6.18, lr=0.000001, samples/sec=8.4, sec/step=0.955, eta=0:04:46
2019-05-20 00:24:23 Iter 199800 [Train]: loss=213.62, epe=6.15, lr=0.000001, samples/sec=8.4, sec/step=0.954, eta=0:03:11
2019-05-20 00:26:01 Iter 199900 [Train]: loss=218.42, epe=6.29, lr=0.000001, samples/sec=8.4, sec/step=0.953, eta=0:01:35
2019-05-20 00:27:40 Iter 200000 [Train]: loss=215.85, epe=6.22, lr=0.000001, samples/sec=8.4, sec/step=0.954, eta=0:00:00
2019-05-20 00:27:40 Iter 200000 [Val]: loss=nan, epe=nan
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcnet.ckpt-200000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcne

## Training log

Here are the training curves for the run above:

![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/loss.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/epe.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/lr.png)

Here are the predictions issued by the model for a few validation samples:

![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val1.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val2.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val3.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val4.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val5.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val6.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val7.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val8.png)